# How to transform Gaia DR2 coordinates into Galactic coordinates

## 0) I just want some "black box" python code...

Make sure you have **astropy** installed and copy-paste the code in the last box at the end of this document.

## 1) Which coordinate system is Gaia DR2 using?

**Reference epoch: 2015.5**

--> This is the point in time at which the star was located at the observed position on the sky.

If the Gaia DR2 has to be cross-matched with a catalog of stars that were observed at a different time (e.g. TGAS: 2015.0), then the proper motion movement of the stars between the different epochs has to be taken into account.

If we use data from Gaia DR2, we investigate the Galaxy's structure/configuration as it was in mid 2015.

**Reference coordinate frame: ICRS**

--> This sets the spatial coordinate frame for positions and proper motions. It defines (ra,dec)=(0,0) and the corresponding coordinate axes and is fixed in time and space. ICRS is independent of Earth's motion. 

ICRS is quite close to the epoch "J2000.0" coordinate frame. This means, (ra,dec)=(0,0) in ICRS is close to the vernal equinox in 2000, and dec~0 corresponds to the equator in year 2000. Since then, equator and equinox have, however, changed their position due to the precession of the Earth's axis, but the ICRS coordinate frame is still the same, i.e. a star that would not move with respect to the Sun still had the same (ra,dec) in 2000 and 2015.

## 2) Which input data do we need?

a) The 3D positions and 3D velocities of the stars:

In [1]:
import numpy as np

#Position of stars:
ra_deg  = np.array([7.7750132145]) #right ascension [rad] in ICRS
dec_deg = np.array([-26.8097293548]) #declination [rad] in ICRS
d_kpc   = np.array([890.547792917])/1000.  #distance from Sun [kpc]

#Velocity of stars:
pm_ra_masyr  = np.array([24.965]) #proper motion in direction of right ascension [mas/yr] in ICRS
pm_dec_masyr = np.array([-9.683]) #proper motion in direction of declination [mas/yr] in ICRS
v_los_kms    = np.array([-4.351]) #line-of-sight velocity [km/s]

b) The position of the Sun in the Galactic plane:

In [2]:
#Galactocentric position of the Sun:
X_GC_sun_kpc = 8.    #[kpc]
Z_GC_sun_kpc = 0.025 #[kpc] (e.g. Juric et al. 2008)

c) The velocity of the Sun with respect to the Local Standard of Rest:

In [3]:
#Velocity of the Sun w.r.t. the Local Standard of Rest (e.g. Schoenrich et al. 2009):
U_LSR_kms = 11.1  # [km/s]
V_LSR_kms = 12.24 # [km/s]
W_LSR_kms = 7.25  # [km/s]

d) The circular velocity of the Galaxy at the Solar radius:

In [4]:
#circular velocity of the Galactic potential at the radius of the Sun:
vcirc_kms = 220. #[km/s] (e.g. Bovy 2015)

--> From this follows as the velocities of the Sun in the Galactocentric inertial frame:

In [5]:
#Galactocentric velocity of the Sun:
vX_GC_sun_kms = -U_LSR_kms           # = -U              [km/s]
vY_GC_sun_kms =  V_LSR_kms+vcirc_kms # = V+v_circ(R_Sun) [km/s]
vZ_GC_sun_kms =  W_LSR_kms           # = W               [km/s]

## 3) Transformation to Galactic sky coordinates:

(l,b) are Galactic coordinates. The Galactic center is at (l,b)=0, and the plane of the disk runs along b=0. (ra,dec) on the plane of the sky needs to be rotated from its orientation with respect to the **[vernal equinox,equator] in 2000.0** (or, more correctly, from the **ICRS**) to (l,b) with respect to **[Galactic center,Galactic plane]**.

**Option A:** Use the galpy package by Jo Bovy (http://galpy.readthedocs.io/en/v1.3.0/, Bovy 2015) together with the "2000.0" coordinate frame. For most applications this is close enough to ICRS.

In [6]:
#Possibility A: Use the fact that epoch "J2000.0" equator/equinox are close to ICRS:

from galpy.util import bovy_coords

#_____(ra,dec) --> Galactic coordinates (l,b):_____
lb = bovy_coords.radec_to_lb(
                ra_deg,dec_deg,
                degree=True,
                epoch=2000.0
                )
l_deg = lb[:,0]
b_deg = lb[:,1]
print "galpy 2000.0:\t\t(l,b) =\t\t",l_deg,b_deg," [deg]"

#_____(pm_ra,pm_dec) --> (pm_l,pm_b)_____
pmlpmb = bovy_coords.pmrapmdec_to_pmllpmbb(
                    pm_ra_masyr,
                    pm_dec_masyr,
                    ra_deg,dec_deg,
                    degree=True,
                    epoch=2000.0
                    )
pml_masyr = pmlpmb[:,0]
pmb_masyr = pmlpmb[:,1]
print "galpy 2000.0:\t\t(pm_l,pm_b) =\t",pml_masyr,pmb_masyr," [mas/yr]"

/Users/trick/software/miniconda2/lib/python2.7/site-packages/astropy/coordinates/sky_coordinate.py:1600: AstropyDeprecationWarning: Passing a frame as a positional argument is now deprecated, use the frame= keyword argument instead.
  "instead.", AstropyDeprecationWarning)

galpy 2000.0:		(l,b) =		[35.79645679] [-85.45759215]  [deg]
galpy 2000.0:		(pm_l,pm_b) =	[-7.39326495] [-25.73618751]  [mas/yr]


**Option B:** If galpy is setup to internally use astropy, it can also deal with ICRS as coordinate frame. For this, the epoch needs to be set to "None".

In [7]:
#Possibility B: Use "None" as epoch in galpy. It should then use internally the astropy ICRS transformation.

#_____(ra,dec) --> Galactic coordinates (l,b):_____
lb = bovy_coords.radec_to_lb(
                ra_deg,dec_deg,
                degree=True,
                epoch=None
                )
l_deg = lb[:,0]
b_deg = lb[:,1]
print "galpy ICRS:\t\t(l,b) =\t\t",l_deg,b_deg," [deg]"

#_____(pm_ra,pm_dec) --> (pm_l,pm_b)_____
pmlpmb = bovy_coords.pmrapmdec_to_pmllpmbb(
                    pm_ra_masyr,
                    pm_dec_masyr,
                    ra_deg,dec_deg,
                    degree=True,
                    epoch=None
                    )
pml_masyr = pmlpmb[:,0]
pmb_masyr = pmlpmb[:,1]
print "galpy ICRS:\t\t(pm_l,pm_b) =\t",pml_masyr,pmb_masyr," [mas/yr]"

galpy ICRS:		(l,b) =		[35.79648236] [-85.45759503]  [deg]
galpy ICRS:		(pm_l,pm_b) =	[-7.39325713] [-25.73618975]  [mas/yr]


**Option C:** Use the rotation matrix from ICRS to Galactic coordinates provided in the Gaia DR2 documentation. See Equation (3.61) here: https://gea.esac.esa.int/archive/documentation/GDR2/pdf/GaiaDR2_documentation_1.0.pdf

In [8]:
#Possibility C: Use the ICRS transformation provided by the Gaia DR2 documentation:
A_G_p = np.array([[-0.0548755604162154,-0.8734370902348850,-0.4838350155487132],
                  [+0.4941094278755837,-0.4448296299600112,+0.7469822444972189],
                  [-0.8676661490190047,-0.1980763734312015,+0.4559837761750669]])

#_____(ra,dec) --> Galactic coordinates (l,b):_____
lb = bovy_coords.radec_to_custom(
                ra_deg,dec_deg,
                T=A_G_p,
                degree=True,
                epoch=None
                )
l_deg = lb[:,0]
b_deg = lb[:,1]
print "Gaia DR2 ICRS (galpy):\t\t(l,b) =\t\t",l_deg,b_deg," [deg]"

#_____(pm_ra,pm_dec) --> (pm_l,pm_b)_____
pmlpmb = bovy_coords.pmrapmdec_to_custom(
                        pm_ra_masyr,pm_dec_masyr,
                        ra_deg,dec_deg,
                        T=A_G_p,
                        degree=True,epoch=2000.0)
pml_masyr = pmlpmb[:,0]
pmb_masyr = pmlpmb[:,1]
print "Gaia DR2 ICRS (galpy):\t\t(pm_l,pm_b) =\t",pml_masyr,pmb_masyr," [mas/yr]"

Gaia DR2 ICRS (galpy):		(l,b) =		[35.7964446] [-85.45759328]  [deg]
Gaia DR2 ICRS (galpy):		(pm_l,pm_b) =	[-7.39327134] [-25.73618567]  [mas/yr]


To illustrate that the galpy black box function is indeed doing the same conversion as the Gaia DR2 documentation suggests in Equations (3.66)-(3.70), here the same calculation done by hand:

In [9]:
#_____(pm_ra,pm_dec) --> (pm_l,pm_b)_____
#space coordinates in rad:
ra_rad  = ra_deg *(np.pi/180.) 
dec_rad = dec_deg*(np.pi/180.) 
l_rad   = l_deg  *(np.pi/180.) 
b_rad   = b_deg  *(np.pi/180.) 

#auxilliary unit vectors:
p_ICRS = np.vstack((-np.sin(ra_rad),np.cos(ra_rad),np.zeros_like(ra_deg))) #Equation (3.64)
q_ICRS = np.vstack((-np.cos(ra_rad)*np.sin(dec_rad),-np.sin(ra_rad)*np.sin(dec_rad),np.cos(dec_rad))) #Equation (3.64)
p_Gal  = np.vstack((-np.sin(l_rad),np.cos(l_rad),np.zeros_like(l_rad))) #Equation (3.65)
q_Gal  = np.vstack((-np.cos(l_rad)*np.sin(b_rad),-np.sin(l_rad)*np.sin(b_rad),np.cos(b_rad))) #Equation (3.65)

#proper motion vector in ICRS:
pm_ICRS_masyr = p_ICRS*pm_ra_masyr + q_ICRS*pm_dec_masyr #Equation (3.66)

#proper motion vector in Galactic coordinates by using the rotation matrix:
pm_Gal_masyr  = np.dot(A_G_p,pm_ICRS_masyr) #Equation (3.68)

#projecting the proper motion vector in (l,b) direction:
pml_masyr = np.squeeze(np.dot(p_Gal.T,pm_Gal_masyr)) #Equation (3.70)
pmb_masyr = np.squeeze(np.dot(q_Gal.T,pm_Gal_masyr)) #Equation (3.70)

print "Gaia DR2 ICRS (by hand):\t\t(pm_l,pm_b) =\t",pml_masyr,pmb_masyr," [mas/yr]"

Gaia DR2 ICRS (by hand):		(pm_l,pm_b) =	-7.393271339713005 -25.736185671100888  [mas/yr]


## 4) Transformation to heliocentric Galactic cartesian coordinates:

The angular 2D position on the plane of the sky (l,b) is now - together with the distance of the star to the Sun - converted to Cartesian coordinates, centered on and co-moving with the Sun. The angular velocities (pm_l,pm_b) are combined with the distance and complemented by the line-of-sight velocity v_los to form a 3D space velocity vector. 

The positions (X,Y,Z) are defined towards the Galactic center, the Galactic North pole, in the direction of Galactic rotation. The corresponding velocities in these coordinate directions are (U,V,W).

In [10]:
#_____(l,b,d) --> Galactic, heliocentric cartesian coordinates (X,Y,Z)_____
xyz = bovy_coords.lbd_to_XYZ(
                l_deg,b_deg,
                d_kpc,
                degree=True)
X_HC_kpc = xyz[:,0]
Y_HC_kpc = xyz[:,1]
Z_HC_kpc = xyz[:,2]

#_____(v_los,pm_l,pm_b) & (l,b,d) --> (vx,vy,vz)______
vxvyvz = bovy_coords.vrpmllpmbb_to_vxvyvz(
                v_los_kms,
                pml_masyr,pmb_masyr,
                l_deg,b_deg,
                d_kpc,
                XYZ=False,degree=True
                )
U_HC_kms = vxvyvz[:,0]
V_HC_kms = vxvyvz[:,1]
W_HC_kms = vxvyvz[:,2]

## 5) Transformation to Galactocentric cylindrical coordinates:

The step from heliocentric to Galactocentric positions and velocities requires assumptions for the position and velocity of the Sun within the Galaxy. See section 2) above.

In [11]:
#______(X,Y,Z) --> Galactic, Galactocentric cylindrical coordinates (R,phi,z)______:
Rzphi= bovy_coords.XYZ_to_galcencyl(
                X_HC_kpc, Y_HC_kpc, Z_HC_kpc, 
                Xsun=X_GC_sun_kpc,Zsun=Z_GC_sun_kpc
                )
R_kpc   = Rzphi[:,0]
phi_rad = Rzphi[:,1]
z_kpc   = Rzphi[:,2]

#______(vx,vy,vz) & (x,y,z) --> (vR,vT,vz)______
vRvTvZ = bovy_coords.vxvyvz_to_galcencyl(
                U_HC_kms, 
                V_HC_kms, 
                W_HC_kms, 
                R_kpc,
                phi_rad, 
                z_kpc,
                Xsun=X_GC_sun_kpc,Zsun=Z_GC_sun_kpc,
                vsun=[vX_GC_sun_kms,vY_GC_sun_kms,vZ_GC_sun_kms], 
                galcen=True
                )
vR_kms = vRvTvZ[:,0]
vT_kms = vRvTvZ[:,1]
vz_kms = vRvTvZ[:,2]

In [12]:
print "Galactic coordinates as calculated by galpy:"
print "R   = ",R_kpc  ,"\t kpc"
print "phi = ",phi_rad,"\t rad"
print "z   = ",z_kpc  ,"\t kpc"
print "v_R = ",vR_kms ,"\t km/s"
print "v_T = ",vT_kms ,"\t km/s"
print "v_z = ",vz_kms ,"\t km/s"

Galactic coordinates as calculated by galpy:
R   =  [7.94567448] 	 kpc
phi =  [0.00519203] 	 rad
z   =  [-0.86292487] 	 kpc
v_R =  [59.52813558] 	 km/s
v_T =  [143.06611879] 	 km/s
v_z =  [3.20086104] 	 km/s


## 6) Doing the conversion in one go with astropy

If you prefer astropy over galpy, the following piece of code demonstrates how to do ICRS coordinate transformations with astropy. Note that the definition of the azimuth phi is different in galpy and astropy.

In [13]:
from astropy import units as u
import astropy.coordinates as coord
import math

#_____coordinate transformation_____
#input coordinates as quantities with units:
ra     = ra_deg           *u.degree
dec    = dec_deg          *u.degree
dist   = d_kpc            *u.kpc
pm_ra  = pm_ra_masyr      *u.mas/u.year
pm_dec = pm_dec_masyr     *u.mas/u.year
v_los  = v_los_kms        *u.km/u.s

#setup frame object:
icrs = coord.ICRS(ra=ra, 
                  dec=dec, 
                  distance=dist, 
                  pm_ra_cosdec=pm_ra, 
                  pm_dec=pm_dec, 
                  radial_velocity=v_los)

#setup representation frame:
gc = coord.Galactocentric(galcen_distance=X_GC_sun_kpc*u.kpc,
                          galcen_v_sun=coord.CartesianDifferential([-vX_GC_sun_kms,vY_GC_sun_kms,vZ_GC_sun_kms]*u.km/u.s),
                          z_sun=Z_GC_sun_kpc*u.kpc)
galcen = icrs.transform_to(gc)

#Galactocentric Cartesian coordinates:
x_kpc = galcen.x.to(u.kpc).value
y_kpc = galcen.y.to(u.kpc).value

#change to Galactocentric cylindrical coordinates:
galcen.set_representation_cls(coord.CylindricalRepresentation,s=coord.CylindricalDifferential)

R_kpc   = galcen.rho.to(u.kpc).value
phi_rad = galcen.phi.to(u.rad).value
z_kpc   = galcen.z.to(u.kpc).value

vR_kms = galcen.d_rho.to(u.km/u.s).value
vT_kms = -(galcen.d_phi.to(u.rad/u.s)*galcen.rho.to(u.km)  / (1.*u.radian)).value
vz_kms = galcen.d_z.to(u.km/u.s).value

print "Galactic coordinates as calculated by astropy:"
print "R   = ",R_kpc  ,"\t kpc"
print "phi = ",phi_rad,"\t rad"
print "z   = ",z_kpc  ,"\t kpc"
print "v_R = ",vR_kms ,"\t km/s"
print "v_T = ",vT_kms ,"\t km/s"
print "v_z = ",vz_kms ,"\t km/s"

Galactic coordinates as calculated by astropy:
R   =  [7.94563548] 	 kpc
phi =  [3.1364006] 	 rad
z   =  [-0.86292487] 	 kpc
v_R =  [59.52813021] 	 km/s
v_T =  [143.06610965] 	 km/s
v_z =  [3.20086404] 	 km/s
